In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
print(tf.__version__)

2.4.1


In [4]:
fashion_mnist = tf.keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()

In [6]:
train_images = train_images/255.0
test_images = test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [9]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [10]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [11]:
tuner_search = RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output',project_name="mnist_fashion")

In [12]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 1 Complete [00h 31m 19s]
val_accuracy: 0.8525000214576721

Best val_accuracy So Far: 0.8525000214576721
Total elapsed time: 00h 31m 19s

Search: Running Trial #2

Hyperparameter    |Value             |Best Value So Far 
conv_1_filter     |80                |96                
conv_1_kernel     |3                 |5                 
conv_2_filter     |48                |64                
conv_2_kernel     |3                 |5                 
dense_1_units     |80                |32                
learning_rate     |0.001             |0.01              

Epoch 1/3
1688/1688 [==============================] - 571s 336ms/step - loss: 0.5111 - accuracy: 0.8182 - val_loss: 0.2876 - val_accuracy: 0.8935
Epoch 2/3
 230/1688 [===>..........................] - ETA: 8:27 - loss: 0.2456 - accuracy: 0.9151

KeyboardInterrupt: 

In [13]:
model=tuner_search.get_best_models(num_models=1)[0]

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 96)        2496      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 64)        153664    
_________________________________________________________________
flatten (Flatten)            (None, 25600)             0         
_________________________________________________________________
dense (Dense)                (None, 32)                819232    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                330       
Total params: 975,722
Trainable params: 975,722
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
  16/1688 [..............................] - ETA: 13:52 - loss: 0.3407 - accuracy: 0.8843

KeyboardInterrupt: 